In [5]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler, MinMaxScaler
import seaborn as sns
import matplotlib.pyplot as plt
import os
import time
from sklearn.utils.class_weight import compute_class_weight
import shap
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, f1_score, precision_score, recall_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from explainerdashboard import InlineExplainer, ExplainerDashboard, ClassifierExplainer
import random

In [9]:
class CorrEncoder:
    """
    CorrEncoder: Takes a dataset as input and uses it for the encode function. Encodes the filtered categories then draws correlations.
    If correlation is above the threshold adds it to a new dataframe then returns the one hot encoded values with the labels.

    Initialisation:
        - data (pd.DataFrame): The Dataset that contains the target column and target label variables.
    """
    
    def __init__(self, data):
        self.data = data.reset_index(drop=True).copy()
        self.data = self.data.drop(columns=['Label'])

    def encode(self, target_column, sparse_n, threshold):
        """
        encode: Takes a target column and target label to encode and draw correlations from. The target column is iterated through
        for all categories that contain more positive values than defined in sparse_n. This allows for filtering of sparse categories.
        The function then one hot encodes the given category with the static target column and draws correlations for them. If correlation
        is greater then threshold then add it to the new DataFrame. The function returns the one hot encoded categories that pass the
        threshold with the target label.

        The purpose of this function is to resolve the high cardinality problem in one hot encoding.

        Parameters:
            - target_column (string): The name of the target column. The target column should contain the various categories to encode.
            - sparse_n (integer): The minimum amount of positive values required for a category after encoding (deals with sparse categories).
            - threshold (float): The threshold for correlation. The function creates onehot encoded columns of all variables that high correlation
              higher that the threshold to the target label.
            - cat (string): The category label to compare to.

        Returns:
            - ohe_df (pd.DataFrame): The one hot encoded values from the target columns.
        """


        self.data[target_column] = self.data[target_column].astype(str)
        value_counts = self.data[target_column].value_counts()
        # Check if number of 1s is above the given threshold set by sparse_n.
        categories = value_counts[value_counts > sparse_n].index.tolist()
        ohe_list = []    
        attack_cat = self.data['attack_cat']
        # Go through each unique category in the target column.
        for c in categories:
            col_name = f'{target_column}_{c}'

            # Create the binary encoding column for the current category and target label
            corr_column = (self.data[target_column] == c).astype(int)
            correlation = corr_column.corr(attack_cat)

            # Check if absolute correlation is greater than threshold.
            if abs(correlation) > threshold:
                corr_column.name = col_name
                ohe_list.append(corr_column)
        print('Number of Encoded Features for', target_column)
        print(len(ohe_list))
        if ohe_list:
            # NOTE: This section can be expanded to include print outs but at the moment am focusing on the evaluations.
            ohe_df = pd.concat(ohe_list, axis=1)
            return ohe_df
        else:
            # This ommits errors (if really high thresholds are used).
            print("No correlations exceed the threshold.")
            return pd.DataFrame()

In [10]:
def get_data(size, rs, threshold, downsample, split_method, scaler_type, category):
    """
    get_data: Example usage of CorrEncoder and Downsampling.

    Parameters:
        - size (float): Proportion split for the validation set.
        - rs (int): Random seeds.
        - threshold (float): Correlation threshold for CorrEncoder().
        - downsample (string, float): Either 'full' or a proportion to downsample the label to.
        - split_method (string): 'slice' or sample the validation set.
        - category (string): The threat category to process data for.
    
    Returns:
        - train_data (pd.DataFrame): The training set.
        - val_data (pd.DataFrame): The validation set.
    """
    feature_names = pd.read_csv('../features2.csv')
    feature_names_list = feature_names['Name'].tolist()
    datasets = []
    for i in range(1, 5):
        df = pd.read_csv(f'../UNSW-NB15_{i}.csv', header=None)
        df.columns = feature_names_list
        df.loc[df['attack_cat'].isnull(), 'attack_cat'] = 'Normal'
        datasets.append(df)
    filtered_datasets = []
    for df in datasets:
        length1 = len(df)
        df.columns = df.columns.str.replace(' ', '')
        df['attack_cat'] = df['attack_cat'].str.replace(r'\s+', '', regex=True)
        df['attack_cat'] = df['attack_cat'].str.replace('Backdoors', 'Backdoor')
        df = df[~df['sport'].astype(str).str.startswith('0x')]
        df = df[~df['dsport'].astype(str).str.startswith('0x')]
        # Does not align with the Features CSV description?
        #df = df.drop(columns=['is_ftp_login'])
        df['ct_ftp_cmd'] = df['ct_ftp_cmd'].astype(str)
        df['ct_ftp_cmd'] = df['ct_ftp_cmd'].replace(' ', 0)
        df['ct_ftp_cmd'] = df['ct_ftp_cmd'].astype(int)
        df['is_ftp_login'] = df['is_ftp_login'].fillna(0)
        df['is_ftp_login'] = np.where(df['is_ftp_login']>1, 1, df['is_ftp_login'])
        df['ct_flw_http_mthd'] = df['ct_flw_http_mthd'].fillna(0).astype(int)
        print(f"Filtered Rows (Cleaning): {length1 - len(df)}")

        df['attack_cat'] = (df['attack_cat'] == category).astype(int)
        if downsample == 'full':
            threat_rows = df[df['attack_cat'] != 'Normal']
            num_threat_rows = len(threat_rows)
            print(df['attack_cat'].value_counts())
            normal_rows = df[df['attack_cat'] == 'Normal']
            sampled_data = normal_rows.sample(n=num_threat_rows, random_state=rs)
            df = pd.concat([threat_rows, sampled_data]).reset_index(drop=True)

        # Downsample by a given pecentage.
        elif downsample is not None:
            mask = (df['Label'].shift(-1) != 1) & (df['Label'].shift(1) != 1)
            normal_rows = df[(df['attack_cat'] == 'Normal') & mask]
            percentage_to_remove = int(len(normal_rows) * downsample)
            rows_to_remove = normal_rows.sample(n=percentage_to_remove, random_state=rs)
            df = df.drop(rows_to_remove.index)
            print(f"Downsampled Rows: {len(rows_to_remove)}")

        filtered_datasets.append(df)
    full_data = pd.concat(filtered_datasets).reset_index(drop=True)

    # These only have a few unique values so easy to encode.
    categorical_columns = ['state', 'service']
    encoder = OneHotEncoder(sparse_output=False, dtype='float32')
    encoded_data = encoder.fit_transform(full_data[categorical_columns])
    encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(categorical_columns), index=full_data.index)
    full_encoded = pd.concat([full_data.drop(columns=categorical_columns), encoded_df], axis=1)
    # Encode categories with many unique values using a threshold.
    encoder = CorrEncoder(full_encoded)
    ohe1 = encoder.encode('dsport', 30, threshold)
    ohe2 = encoder.encode('proto', 30, threshold)
    ohe3 = encoder.encode('sport', 30, threshold)
    ohe4 = encoder.encode('srcip', 30, threshold)
    ohe5 = encoder.encode('dstip', 30, threshold)
    cols_to_drop = ['dsport', 'proto', 'sport', 'srcip', 'dstip']
    filtered_data = full_encoded.drop(columns=cols_to_drop)
    combined_data = pd.concat([filtered_data, ohe1, ohe2, ohe3, ohe4, ohe5], axis=1)
    df_features = combined_data.drop(columns=['attack_cat', 'Label'])
    if scaler_type == 'minmax':
        scaler = MinMaxScaler()
    elif scaler_type == 'standard':
        scaler = StandardScaler()
    scaled_data = scaler.fit_transform(df_features)
    final_data = pd.DataFrame(scaled_data, columns=df_features.columns, index=combined_data.index)
    final_data['attack_cat'] = combined_data['attack_cat']
    if split_method == 'slice':
        slice_size = int(size * len(final_data))
        val_start = random.randrange(0, len(final_data) - 2 * slice_size)
        val_end = val_start + slice_size
        val_data = final_data.iloc[val_start:val_end]
        train_data = final_data.drop(val_data.index)
    elif split_method == 'shuffle':
        train_data, val_data = train_test_split(final_data, test_size=size, random_state=rs)
    
    return train_data, val_data


# 20% Downsample

In [ ]:
train_data, val_data = get_data(
    size=0.2,
    rs=42,
    threshold=0.1,
    downsample=0.99,
    split_method='shuffle',
    scaler_type='standard',
    category='Analysis'
    )

C:\Users\adi_s\AppData\Local\Temp\ipykernel_28368\1927573160.py:21: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
C:\Users\adi_s\AppData\Local\Temp\ipykernel_28368\1927573160.py:21: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.


Filtered Rows (Cleaning): 62
Downsampled Rows: 0
Filtered Rows (Cleaning): 61
Downsampled Rows: 0
Filtered Rows (Cleaning): 105
Downsampled Rows: 0
Filtered Rows (Cleaning): 75
Downsampled Rows: 0
Number of Encoded Features for dsport
1
Number of Encoded Features for proto
1


In [ ]:
print(train_data['attack_cat'].value_counts())
print(val_data['attack_cat'].value_counts())

attack_cat
0    896588
1     19443
Name: count, dtype: int64
attack_cat
0    224205
1      4803
Name: count, dtype: int64


In [ ]:
X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_val = val_data.drop(columns=['attack_cat'])
y_val = val_data['attack_cat']
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.9956027737022287
Precision: 0.8834343434343435
Recall: 0.9104726212783677
F1 Score: 0.8967497180354763

Classification Report:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00    224205
           1       0.88      0.91      0.90      4803

    accuracy                           1.00    229008
   macro avg       0.94      0.95      0.95    229008
weighted avg       1.00      1.00      1.00    229008



# 90% Downsample

In [ ]:
train_data, val_data = get_data(
    size=0.2,
    rs=42,
    threshold=0.1,
    downsample=0.9,
    split_method='shuffle',
    scaler_type='standard',
    category='Analysis'
    )

C:\Users\adi_s\AppData\Local\Temp\ipykernel_17628\1339685965.py:21: DtypeWarning: Columns (1,3,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'UNSW-NB15_{i}.csv', header=None)
C:\Users\adi_s\AppData\Local\Temp\ipykernel_17628\1339685965.py:21: DtypeWarning: Columns (3,39,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(f'UNSW-NB15_{i}.csv', header=None)


Filtered Rows (Cleaning): 62
Downsampled Rows: 606951
Filtered Rows (Cleaning): 61
Downsampled Rows: 578493
Filtered Rows (Cleaning): 105
Downsampled Rows: 479475
Filtered Rows (Cleaning): 75
Downsampled Rows: 310883
Number of Encoded Features for dsport
9
Number of Encoded Features for proto
1
Number of Encoded Features for sport
2
Number of Encoded Features for srcip
4
Number of Encoded Features for dstip
10


In [ ]:
print(train_data['attack_cat'].value_counts())
print(val_data['attack_cat'].value_counts())

attack_cat
0    194117
1     19407
Name: count, dtype: int64
attack_cat
0    48542
1     4839
Name: count, dtype: int64


In [ ]:
X_train = train_data.drop(columns=['attack_cat'])
y_train = train_data['attack_cat']
X_val = val_data.drop(columns=['attack_cat'])
y_val = val_data['attack_cat']
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)
y_pred = rf_classifier.predict(X_val)
print("Accuracy:", accuracy_score(y_val, y_pred))
print("Precision:", precision_score(y_val, y_pred))
print("Recall:", recall_score(y_val, y_pred))
print("F1 Score:", f1_score(y_val, y_pred))
print("\nClassification Report:\n", classification_report(y_val, y_pred))

Accuracy: 0.9892658436522358
Precision: 0.9003378378378378
Recall: 0.9913205207687539
F1 Score: 0.9436411920920625

Classification Report:
               precision    recall  f1-score   support

           0       1.00      0.99      0.99     48542
           1       0.90      0.99      0.94      4839

    accuracy                           0.99     53381
   macro avg       0.95      0.99      0.97     53381
weighted avg       0.99      0.99      0.99     53381

